data structure:  
- notesShet  
   - image (obrazek z arkuszem)
    - accolade[] (po dwie pieciolinie)  
        - image (obrazek z akolada)
        - line[] -(dwie linie)
            - image (obrazek z linia)
            - note[]
                - image (obrazek z nuta)
                - sound (dzwiek **ten zapis trzeba ustalic**)
                - countRows (ilosc czarnych pikseli w wierszach)
                - countColumns (ilosc czarnych pikseli w kolumnach)
                - ratioLeft (Stosunek zaciemninych pikseli po lewej do ilosci pikseli w ogonku)
                - ratioRight (Stosunek zaciemninych pikseli po prawej do ilosci pikseli w ogonku)
                - ratioLR (Stosunek zaciemnionych pikseli po obu stronach ogonka)
                - type = 0 (Typ nuty  
                                    + 0 - nie sprawdzona 
                                    + 999 - takt 
                                    + -1 - nie rozpoznana 
                                    + 1 - cala 
                                    + 2 - polnota 
                                    + 4 - cwiercnuta 
                                    + 404 - cztery cwiartki etc.)
        - trebleLine = line[0]
        - bassLine = line[1]

In [21]:

%%writefile NotesShet.py

#i_picture - obraz wejsciowy (macierz 2d)
def countBlackPixels(i_picture):
    height, width = i_picture.shape
    rowsBlackPixelCount = numpy.zeros(shape=(height,1),dtype=numpy.int16)
    columnsBlacPixelsCount = numpy.zeros(shape=(width,1),dtype=numpy.int16)
    
    for iRow in range(0,height):
        for iCol in range(0, width):
            if (i_picture[iRow,iCol]<=127):
                rowsBlackPixelCount[iRow]=rowsBlackPixelCount[iRow]+1
                    
    for iCol in range(0,width):
        for iRow in range(0,height):
            if (i_picture[iRow,iCol]<=127):
                columnsBlacPixelsCount[iCol]=columnsBlacPixelsCount[iCol]+1
                
    return [rowsBlackPixelCount, columnsBlacPixelsCount]
                
    

class Note:
    #image
    #sound
    #countRows
    #countColumns
    #ratioLeft
    #ratioRight
    #ratioLR
    #type
            
    def __init__(self, i_picture):
        self.image = picture
        height, width = i_picture.shape
        self.countRows, self.countColumns = countBlackPixels(self.image)
        self.sound=0
        self.ratioLeft=0
        self.ratioRight=0
        self.ratioLR=0
        self.type=0
        
class Line:
    #image
    note = []
    
    def __init__(self, i_picture):
        self.image = i_picture
        height, width = i_picture.shape
        rowsBlackPixelCount, columnsBlacPixelsCount = countBlackPixels(self.image)
        
        notesLocations = []
        begin = 0
        pixelVal = math.ceil(numpy.percentile(columnsBlacPixelsCount,25)) #math.ceil(numpy.mean(columnsBlacPixelsCount)) +1
    
        tmp=[0,0]
    
        for iCol in range(1,width):
            if((columnsBlacPixelsCount[iCol]>pixelVal) and (columnsBlacPixelsCount[iCol-1]<=pixelVal)):
                #tmp[0]=iCol
                notesLocations.append(iCol)
            if((columnsBlacPixelsCount[iCol-1]>pixelVal) and (columnsBlacPixelsCount[iCol]<=pixelVal)):
                #tmp[1]=iCol
                notesLocations.append(iCol)
                
        for iNote in range(0,len(notesLocations),2):
            tmp=self.image[0:height,(notesLocations[iNote]-1):(notesLocations[iNote+1]+1)]
            self.note.append(tmp)

class Accolade:
    #image
    line = []
    
    def __init__(self, i_pictureWhole, i_pictureTreble, i_pictureBass):

        self.image = i_pictureWhole
        self.line.append(Line(i_pictureTreble))
        self.line.append(Line(i_pictureBass))        
        self.trebleLine = self.line[0]
        self.bassLine = self.line[1]
        
    
class NotesShet:
    #image
    accolade = []
    
    def __init__(self, i_picture):
        self.image = i_picture
        height, width = i_picture.shape
        rowsBlackPixelCount, columnsBlacPixelsCount = countBlackPixels(self.image)
        
        linesLocation = []
        black = 0
        for iRow in range(0,height):
            if(rowsBlackPixelCount[iRow]>(width*0.5)):
                if(black==0):
                    linesLocation.append(iRow)
                    black=1
            else:
                black=0
    
        lineSpaces = linesLocation[1]-linesLocation[0]
        #lineSpaces = 0
    
        
        for iAccolade in range(0,len(linesLocation),10):
            pictureWhole=self.image[(linesLocation[iAccolade+0]-2*lineSpaces):(linesLocation[iAccolade+9]+2*lineSpaces),0:width]
            pictureTreble=self.image[(linesLocation[iAccolade+0]-2*lineSpaces):(linesLocation[iAccolade+4]+2*lineSpaces),0:width]
            pictureBass = self.image[(linesLocation[iAccolade+5]-2*lineSpaces):(linesLocation[iAccolade+9]+2*lineSpaces),0:width]
            self.accolade.append(Accolade(pictureWhole,pictureTreble,pictureBass))
        
        


Overwriting NotesShet.py


In [1]:


def cutLines(imgIn, plots):
    height, width = imgIn.shape
    rowsForPlot = numpy.zeros(shape=(height,width))
    rowsBlackPixelCount = numpy.zeros(shape=(height,1),dtype=numpy.int16)
    
    for iRow in range(0,height):
        for iCol in range(0, width):
            if (imgIn[iRow,iCol]<=127):
                rowsBlackPixelCount[iRow]=rowsBlackPixelCount[iRow]+1
                if(plots==1):
                    rowsForPlot[iRow,rowsBlackPixelCount[iRow]-1]=254
    
    if(plots==1):
        plt.rcParams["figure.figsize"] = [18.0, 24]
        image = numpy.concatenate((imgIn,rowsForPlot),axis=1)
        plt.imshow(image,cmap='gray' )
        plt.show()

    
    linesLocation = []
    black = 0
    for iRow in range(0,height):
        if(rowsBlackPixelCount[iRow]>(width*0.5)):
            if(black==0):
                linesLocation.append(iRow)
                black=1
        else:
            black=0
    
    lineSpaces = linesLocation[1]-linesLocation[0]
    #lineSpaces = 0
    
    accoladeS = []
    for iAccolade in range(0,len(linesLocation),10):
        accoladeS.append(imgIn[(linesLocation[iAccolade+0]-2*lineSpaces):(linesLocation[iAccolade+4]+2*lineSpaces),0:width])
        accoladeS.append(imgIn[(linesLocation[iAccolade+5]-2*lineSpaces):(linesLocation[iAccolade+9]+2*lineSpaces),0:width])   
    
    return(accoladeS)


In [2]:
def cutSingleNotes(imgIn, plots):
    height, width = imgIn.shape
    columnsForPlot = numpy.zeros(shape=(height,width))
    columnsBlacPixelsCount = numpy.zeros(shape=(width,1),dtype=numpy.int16)
    
    for iCol in range(0,width):
        for iRow in range(0,height):
            if (imgIn[iRow,iCol]<=127):
                columnsBlacPixelsCount[iCol]=columnsBlacPixelsCount[iCol]+1
                if(plots==1):
                    columnsForPlot[columnsBlacPixelsCount[iCol]-1,iCol]=254  
    
    if(plots==1):
        plt.rcParams["figure.figsize"] = [18.0, 24]
        image = numpy.concatenate((imgIn,columnsForPlot),axis=0)
        plt.imshow(image,cmap='gray' )
        plt.show()
        #print(columnsBlacPixelsCount)
    
    notesLocations = []
    begin = 0
    pixelVal = math.ceil(numpy.percentile(columnsBlacPixelsCount,25)) #math.ceil(numpy.mean(columnsBlacPixelsCount)) +1
    
    tmp=[0,0]
    
    for iCol in range(1,width):
        if((columnsBlacPixelsCount[iCol]>pixelVal) and (columnsBlacPixelsCount[iCol-1]<=pixelVal)):
            #tmp[0]=iCol
            notesLocations.append(iCol)
        if((columnsBlacPixelsCount[iCol-1]>pixelVal) and (columnsBlacPixelsCount[iCol]<=pixelVal)):
            #tmp[1]=iCol
            notesLocations.append(iCol)
            
        #if((tmp[0]>0)&&(tmp[1]>0)):
        #    notesLocations.append(tmp);
        #    tmp=[0,0]
        
    
    notes=[]
    for iNote in range(0,len(notesLocations),2):
        tmp=imgIn[0:height,(notesLocations[iNote]-1):(notesLocations[iNote+1]+1)]
        notes.append(tmp)
        if(plots==1):
            plt.rcParams["figure.figsize"] = [2, 3]
            plt.imshow(tmp,cmap='gray' )
            plt.show()
    

In [20]:
import numpy
import cv2
import math
from matplotlib import pyplot as plt

# Load an color image in grayscale
img_l = cv2.imread('notes001.jpg')
img_t = cv2.cvtColor(img_l, cv2.COLOR_BGR2GRAY)
tmp, img = cv2.threshold(img_t,200,255,cv2.THRESH_BINARY)

ns = NotesShet(img)





In [4]:
plt.rcParams["figure.figsize"] = [18.0, 12.0]
plt.imshow(out[0],cmap='gray' )
plt.show()



NameError: name 'out' is not defined